In [3]:
import mediapipe as mp
import cv2
import numpy as np
import math

from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

mpHands = mp.solutions.hands
mpDraw = mp.solutions.drawing_utils

In [2]:
def findHandLandMarks(image, handNumber, draw=False):
    originalImage = image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # mediapipe needs RGB
    results = hands.process(image)
    landMarkList = []

    if results.multi_hand_landmarks:  # returns None if hand is not found
        hand = results.multi_hand_landmarks[handNumber] #results.multi_hand_landmarks returns landMarks for all the hands

        for id, landMark in enumerate(hand.landmark):
            # landMark holds x,y,z ratios of single landmark
            imgH, imgW, imgC = originalImage.shape  # height, width, channel for image
            xPos, yPos = int(landMark.x * imgW), int(landMark.y * imgH)
            landMarkList.append([id, xPos, yPos])

        if draw:
            mpDraw.draw_landmarks(originalImage, hand, mpHands.HAND_CONNECTIONS)

    return landMarkList

In [4]:
hands = mpHands.Hands(max_num_hands=2, min_detection_confidence=0.7,
                                   min_tracking_confidence=0.7)

webcamFeed = cv2.VideoCapture(0)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
print(volume.GetVolumeRange()) 

(-63.5, 0.0, 0.5)


In [5]:
while True:
    status, image = webcamFeed.read()
    handLandmarks1 = findHandLandMarks(image=image, handNumber=0, draw=True)
    try:
        handlandmarks2 = findHandLandMarks(image=image, handNumber=1, draw=True)
    except:
        pass

    if(len(handLandmarks1) != 0 and len(handlandmarks2) != 0):
        #for volume control we need 4th and 8th landmark
        x11, y11 = handLandmarks1[4][1], handLandmarks1[4][2]
        x21, y21 = handLandmarks1[8][1], handLandmarks1[8][2]
        length = math.hypot(x21-x11, y21-y11)
        print(length)

        x12, y12 = handlandmarks2[4][1], handlandmarks2[4][2]
        x22, y22 = handlandmarks2[8][1], handlandmarks2[8][2]
        length2 = math.hypot(x22-x12, y22-y12)

        #Hand range(length): 50-250
        #Volume Range: (-65.25, 0.0)
        if(length>30 and length<200 and length2<25):
            volumeValue = np.interp(length, [27, 200], [-62, 0.0]) #coverting length to proportionate to volume range
            volume.SetMasterVolumeLevel(volumeValue, None)


        cv2.circle(image, (x11, y11), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(image, (x21, y21), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(image, (x11, y11), (x21, y21), (255, 0, 255), 3)
        cv2.circle(image, (x12, y12), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(image, (x22, y22), 15, (255, 0, 255), cv2.FILLED)        
        cv2.line(image, (x12, y12), (x22, y22), (255, 0, 255), 3)

    cv2.imshow("Volume", cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
webcamFeed.release()

107.64757312638311
31.304951684997057
42.37924020083418
37.73592452822641
40.311288741492746
55.31726674375732
41.182520563948
40.459856648287825
33.24154027718932
33.30165161069342
34.828149534536
32.64965543462902
35.35533905932738
160.0781059358212
197.0406049523803
283.8203657245195
337.6536687198882
363.91757308489514
398.2825630127435
406.6165269636737
432.77014684471936
439.9022618718845
437.7682035050056
433.4847171469832
423.557552169714
243.57750306627253
179.8777362543792
47.80167361086848
49.92995093127972
34.539832078341085
187.62729012593024
334.5205524328811
398.2072324807776
351.0099713683359
144.42298985964803
74.41102068914255
31.400636936215164
216.0208323287363
268.1510768205118
382.35062442737035
392.16833120485387
361.99585632987566
99.80981915623332
36.24913792078371
193.38303958723992
370.0283772901749
386.3586416789458
395.50094816574085
76.66159403508382
25.63201123595259
29.61418578992169
26.476404589747453
277.6508598942204
390.1512527213004
405.097519123482